# A summary of multi-region results

Summarize geotagged tweets and generated visits of the multiple regions.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import yaml
from scipy import stats

def load_region_tweets(region=None):
    df = pd.read_csv(f'../../dbs/{region}/geotweets.csv')
    num_users = len(df.userid.unique())
    num_geo = len(df)
    return region, num_users, num_geo


def user_proc(data):
    return pd.Series({'pkt': data.pkt.sum(),
                      'num_trips': data.num_trip.sum()})


def quantify_visits(region=None, runid=1):
    df_v = pd.read_csv(f'../../dbs/{region}/visits/visits_{runid}_stats.csv')
    df_users = df_v.groupby('userid').apply(user_proc).reset_index()
    return region, df_users.pkt.sum(), df_users.num_trips.sum()

def get_recent_records_gt(data):
    data = data.loc[data['Year'] == max(data['Year']), ['Year', 'Unit', 'Value']].reset_index(drop=True)
    return data

region_list = ['sweden', 'netherlands', 'saopaulo', 'australia', 'austria', 'barcelona',
               'capetown', 'cebu', 'egypt', 'guadalajara', 'jakarta',
               'johannesburg', 'kualalumpur', 'lagos', 'madrid', 'manila', 'mexicocity', 'moscow', 'nairobi',
               'rio', 'saudiarabia', 'stpertersburg', 'surabaya']

region_list2 = ['australia', 'austria', 'barcelona', 'capetown', 'cebu', 'egypt', 'guadalajara', 'jakarta',
               'johannesburg', 'kualalumpur', 'lagos', 'madrid', 'manila', 'mexicocity', 'moscow', 'nairobi',
               'rio', 'saudiarabia', 'stpertersburg', 'surabaya']

with open('../../lib/regions.yaml') as f:
    region_manager = yaml.load(f, Loader=yaml.FullLoader)

## 1 Summarize the geotagged tweets used as input to the model
Geotagged tweets: No. of geotagged tweets, No. of Twitter users.

Generated visits: No. of trips, PKT

Days in total is 120, therefore, it needs to scale up by multiplying 365/120 to represent all the weekday trips in a year.

In [3]:
df = pd.DataFrame([load_region_tweets(region=x) for x in region_list],
                  columns=('region', 'num_users', 'num_geotweets'))

## 2 Explore the visits generated from the model

In [4]:
df_v = pd.DataFrame([quantify_visits(region=x) for x in region_list],
                    columns=('region', 'pkt', 'num_trips'))
df_v.loc[:, 'pkt_yr'] = df_v.loc[:, 'pkt']/10e6 /120 * 365
df_v.loc[:, 'num_trips_yr'] = df_v.loc[:, 'num_trips']/10e6 /120 * 365

## 3 Combine different dimensions

In [5]:
df_v.loc[:, 'gdp_capita'] = df_v.loc[:, 'region'].apply(lambda x: region_manager[x]['gdp_capita'])
df_v.loc[:, 'region_name'] = df_v.loc[:, 'region'].apply(lambda x: region_manager[x]['name'])
df_v.loc[:, 'country'] = df_v.loc[:, 'region'].apply(lambda x: region_manager[x]['country'])
df_v.loc[:, 'pop_country'] = df_v.loc[:, 'region'].apply(lambda x: region_manager[x]['pop_country'])

In [6]:
df_n = pd.merge(df, df_v, on='region', how='inner')
df_n.loc[:, 'pkt_yr_capita'] = df_n.loc[:, 'pkt_yr'] / df_n.loc[:, 'num_users']

## 4 Compare with the 'ground truth' of PKT

In [7]:
df_p = pd.read_csv('../../dbs/T000_input.txt')
df_p = df_p.loc[df_p['Variable'] == 'Total inland passenger transport']
df_p = df_p.groupby('Country').apply(get_recent_records_gt).reset_index().drop(columns=['level_1'])

In [8]:
df_n = pd.merge(df_n, df_p, left_on='country', right_on='Country', how='outer').dropna(subset=['region'])
df_n.loc[:, 'pkt_yr_capita_gt'] = df_n.apply(lambda row: row['Value']/row['pop_country']/10e6 if not np.isnan(row['Value']) else np.nan,
                                             axis=1)# million / capita /yr
df_n = df_n.drop(columns=['Country', 'Unit', 'Value'])

In [11]:
df_n.to_csv('../../results/multi-region_stats.csv', index=False, encoding = "utf-8")

## 5 Correlation between gdp_capita and pkt_yr_capita

In [9]:
stats.pearsonr(df_n.loc[:, 'pkt_yr_capita'], df_n.loc[:, 'gdp_capita'])

(0.6496584687712351, 0.0007938698847068716)